In [17]:
from sklearn import datasets
data_breast_cancer = datasets.load_breast_cancer(as_frame=True)

In [18]:
from sklearn.model_selection import train_test_split

X = data_breast_cancer.data
y = data_breast_cancer.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
# Zbuduj ensemble używając klasyfikatorów binarnych, których używałeś(aś) w poprzednich ćwiczeniach, tj.: drzewa decyzyjne, regresja logistyczna, k najbliższych sąsiadów, do klasyfikacji w oparciu o cechy: mean texture, mean symmetry. Użyj domyślnych parametrów.
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

dt_clf = DecisionTreeClassifier()
log_clf = LogisticRegression()
knn_clf = KNeighborsClassifier()

voting_clf_hard = VotingClassifier(
    estimators=[('dt', dt_clf), ('log', log_clf), ('knn', knn_clf)],
    voting='hard'
)

voting_clf_soft = VotingClassifier(
    estimators=[('dt', dt_clf), ('log', log_clf), ('knn', knn_clf)],
    voting='soft'
)


In [20]:
# Porównaj dokładność (accuracy) w/w klasyfikatorów z zespołem z głosowaniem typu hard oraz soft.
# Zapisz rezultaty jako listę par (dokładność_dla_zb_uczącego, dokładność_dla_zb_testującego) dla każdego z w/w klasyfikatorów (razem 5 elementów) i umieść ją w pliku Pickle o nazwie acc_vote.pkl
# Zapisz klasyfikatory jako listę w pliku Pickle o nazwie vote.pkl
from sklearn.metrics import accuracy_score
import pickle

acc_vote = []

for clf in (dt_clf, log_clf, knn_clf, voting_clf_hard, voting_clf_soft):
    clf.fit(X_train[['mean texture', 'mean symmetry']], y_train)
    y_pred = clf.predict(X_test[['mean texture', 'mean symmetry']])
    acc_vote.append((accuracy_score(y_train, clf.predict(X_train[['mean texture', 'mean symmetry']])), accuracy_score(y_test, y_pred)))
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

with open('acc_vote.pkl', 'wb') as f:
    pickle.dump(acc_vote, f)

with open('vote.pkl', 'wb') as f:
    pickle.dump([dt_clf, log_clf, knn_clf, voting_clf_hard, voting_clf_soft], f)

DecisionTreeClassifier 0.631578947368421
LogisticRegression 0.7017543859649122
KNeighborsClassifier 0.6403508771929824
VotingClassifier 0.7017543859649122
VotingClassifier 0.6666666666666666


In [21]:
# Wykonaj na zbiorze uczącym wykorzystując 30 drzew decyzyjnych:
# • Bagging,
# • Bagging z wykorzystaniem 50% instancji,
# • Pasting,
# • Pasting z wykorzystaniem 50% instancji, oraz • Random Forest,
# • AdaBoost,
# • Gradient Boosting.
# Oblicz dokładności oraz zapisz je jako listę par (dokładność_dla_zb_uczącego, dokładność_dla_zb_testującego) dla każdego z w/w estymatorów (razem 7 elementów) w pliku Pickle o nazwie acc_bag.pkl.
# Zapisz klasyfikatory jako listę w pliku Pickle o nazwie bag.pkl

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, random_state=42)
bag_clf_50 = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, max_samples=0.5, random_state=42)
pas_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, bootstrap=False, random_state=42)
pas_clf_50 = BaggingClassifier(DecisionTreeClassifier(), n_estimators=30, bootstrap=False, max_samples=0.5, random_state=42)
rf_clf = RandomForestClassifier(n_estimators=30, random_state=42)
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(), n_estimators=30, random_state=42)
grad_clf = GradientBoostingClassifier(n_estimators=30, random_state=42)

acc_bag = []

for clf in (bag_clf, bag_clf_50, pas_clf, pas_clf_50, rf_clf, ada_clf, grad_clf):
    clf.fit(X_train[['mean texture', 'mean symmetry']], y_train)
    y_pred = clf.predict(X_test[['mean texture', 'mean symmetry']])
    acc_bag.append((accuracy_score(y_train, clf.predict(X_train[['mean texture', 'mean symmetry']])), accuracy_score(y_test, y_pred)))
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

with open('acc_bag.pkl', 'wb') as f:
    pickle.dump(acc_bag, f)

with open('bag.pkl', 'wb') as f:
    pickle.dump([bag_clf, bag_clf_50, pas_clf, pas_clf_50, rf_clf, ada_clf, grad_clf], f)




BaggingClassifier 0.6754385964912281
BaggingClassifier 0.6842105263157895
BaggingClassifier 0.6228070175438597
BaggingClassifier 0.6491228070175439
RandomForestClassifier 0.6754385964912281
AdaBoostClassifier 0.631578947368421
GradientBoostingClassifier 0.7105263157894737


In [35]:
# Przeprowadź sampling 2 cech z wszystkich dostepnych bez powtórzeń z wykorzystaniem 30 drzew decyzyjnych, wybierz połowę instancji dla każdego z drzew z powtórzeniami.
# Zapisz dokładności w/w estymatora jako listę : dokładność_dla_zb_uczącego, dokładność_dla_zb_testującego w pliku Pickle acc_fea.pkl.
# Zapisz klasyfikator jako jednoelementową listę w pliku Pickle o nazwie fea.pkl

bag_clf_fea = BaggingClassifier(n_estimators=30, max_features=2, max_samples=0.5, bootstrap_features=False, random_state=42)

bag_clf_fea.fit(X_train, y_train)
y_pred = bag_clf_fea.predict(X_test)
acc_fea = [accuracy_score(y_train, bag_clf_fea.predict(X_train)), accuracy_score(y_test, y_pred)]
print(bag_clf_fea.__class__.__name__, accuracy_score(y_test, y_pred))
print(bag_clf)
print(bag_clf_fea)
with open('acc_fea.pkl', 'wb') as f:
    pickle.dump(acc_fea, f)

with open('fea.pkl', 'wb') as f:
    pickle.dump([bag_clf_fea], f)

BaggingClassifier 0.9736842105263158
BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=30,
                  random_state=42)
BaggingClassifier(max_features=2, max_samples=0.5, n_estimators=30,
                  random_state=42)


In [32]:
# Sprawdź, które cechy dają najwięszą dokładność. Dostęp do poszczególnych estymatorów, aby obliczyć dokładność, możesz uzyskać za pomocą: BaggingClasifier.estimators_, cechy wybrane przez sampling dla każdego z estymatorów znajdziesz w: BaggingClassifier.estimators_features_. Zbuduj ranking estymatorów jako DataFrame, który będzie mieć w kolejnych kolumnach: dokładność dla zb. uczącego, dokładnośc dla zb. testującego, lista nazw cech. Każdy wiersz to informacje o jednym estymatorze. DataFrame posortuj malejąco po wartościach dokładności dla zbioru testującego i uczącego oraz zapisz w pliku Pickle o nazwie acc_fea_rank.pkl

import pandas as pd

acc_fea_rank = []

for estimator, features in zip(bag_clf_fea.estimators_, bag_clf_fea.estimators_features_):
    estimator.fit(X_train.iloc[:, features], y_train)
    y_pred = estimator.predict(X_test.iloc[:, features])
    acc_fea_rank.append((accuracy_score(y_train, estimator.predict(X_train.iloc[:, features])), accuracy_score(y_test, y_pred), X_train.columns[features].tolist()))

acc_fea_rank_df = pd.DataFrame(acc_fea_rank, columns=['train_accuracy', 'test_accuracy', 'features'])
acc_fea_rank_df = acc_fea_rank_df.sort_values(by=['train_accuracy', 'test_accuracy'], ascending=False)

with open('acc_fea_rank.pkl', 'wb') as f:
    pickle.dump(acc_fea_rank_df, f)


,train_accuracy,test_accuracy,features
1,1.0,0.938596,"[mean radius, mean concavity]"
20,1.0,0.921053,"[worst perimeter, symmetry error]"
6,1.0,0.912281,"[worst perimeter, mean smoothness]"
15,1.0,0.903509,"[worst fractal dimension, mean area]"
13,1.0,0.894737,"[perimeter error, worst radius]"
